In [4]:
import pandas as pd
from sklearn.cluster import DBSCAN
import json
import os

In [9]:
os.path.dirname(os.path.dirname(os.getcwd()))

'c:\\Users\\Yusuf\\Desktop\\GA-Tech\\CSE6242\\Project\\Koto_Repo\\telematics'

In [10]:
path = "c:\\Users\\Yusuf\\Desktop\\GA-Tech\\CSE6242\\Project\\Koto_Repo\\telematics\\"

In [11]:
#Mongo Unpacker
data = [json.loads(line) for line in open(path+'dwellTimes.json', 'r')]

In [30]:
df = pd.DataFrame(columns = ['seq', 'start_timestamp', 'end_timestamp','vin'])
column_names_dict = columns={'index':'seq',0:'start_timestamp',1:"end_timestamp"}
for i in range(len(data)):
     temp_df = pd.DataFrame(data[i]['dwell']).transpose().reset_index().rename(columns=column_names_dict)
     temp_df['vin'] = data[i]["_id"]
     df = df.append(temp_df, ignore_index=True)

In [32]:
df.head()

,seq,start_timestamp,end_timestamp,vin
0,1338,2020-01-09 19:00:00 EST,2020-01-09 19:33:29 EST,070aac90eee9f2eaf444a1ccbb894df0
1,1340,2020-01-09 20:35:48 EST,2020-01-09 22:11:36 EST,070aac90eee9f2eaf444a1ccbb894df0
2,1341,2020-01-10 05:19:27 EST,2020-01-10 05:25:25 EST,070aac90eee9f2eaf444a1ccbb894df0
3,1343,2020-01-10 05:36:52 EST,2020-01-10 05:40:39 EST,070aac90eee9f2eaf444a1ccbb894df0
4,1344,2020-01-10 09:34:54 EST,2020-01-10 11:59:28 EST,070aac90eee9f2eaf444a1ccbb894df0


In [41]:
df2 = df.copy()

In [42]:
df2 = df2.astype({'seq':'int64'})
df2['start_timestamp'] = pd.to_datetime(df2['start_timestamp'])
df2['end_timestamp'] = pd.to_datetime(df2['end_timestamp'])

In [54]:
df2['arrival_hour'] = df2["start_timestamp"].dt.hour
df2['arrival_date'] = df2["start_timestamp"].dt.date
df2['arrival_day'] = df2["start_timestamp"].dt.day_name()
df2['trip_time'] = df2['end_timestamp'] - df2['start_timestamp']
df2 = df2.sort_values(by =['vin','seq'])

In [65]:
df2["next_trip_start_timestamp"] = df2["start_timestamp"].shift(periods=-1)
df2["next_trip_seq"] = df2['seq'].shift(periods=-1)
df2["dwell_time"] = df2['next_trip_start_timestamp'] - df2['end_timestamp']
df2['seq_diff'] = df2['next_trip_seq'] - df2['seq']

In [66]:
df2.head(20)

,seq,start_timestamp,end_timestamp,vin,arrival_hour,arrival_date,arrival_day,trip_time,next_trip_start_timestamp,next_trip_seq,dwell_time,seq_diff
35515,38,2020-01-12 22:02:17,2020-01-12 22:11:49,0001556fa4a96efdfb0e6f9c079d2fa5,22,2020-01-12,Sunday,0 days 00:09:32,2020-01-13 12:21:14,40.0,0 days 14:09:25,2.0
35516,40,2020-01-13 12:21:14,2020-01-13 12:28:51,0001556fa4a96efdfb0e6f9c079d2fa5,12,2020-01-13,Monday,0 days 00:07:37,2020-01-13 14:02:56,41.0,0 days 01:34:05,1.0
35517,41,2020-01-13 14:02:56,2020-01-13 14:10:37,0001556fa4a96efdfb0e6f9c079d2fa5,14,2020-01-13,Monday,0 days 00:07:41,2020-01-13 14:17:32,42.0,0 days 00:06:55,1.0
35518,42,2020-01-13 14:17:32,2020-01-13 14:26:23,0001556fa4a96efdfb0e6f9c079d2fa5,14,2020-01-13,Monday,0 days 00:08:51,2020-01-13 14:49:29,43.0,0 days 00:23:06,1.0
35519,43,2020-01-13 14:49:29,NaT,0001556fa4a96efdfb0e6f9c079d2fa5,14,2020-01-13,Monday,NaT,2020-01-13 15:34:58,44.0,NaT,1.0
35520,44,2020-01-13 15:34:58,2020-01-13 15:38:32,0001556fa4a96efdfb0e6f9c079d2fa5,15,2020-01-13,Monday,0 days 00:03:34,2020-01-13 17:54:20,45.0,0 days 02:15:48,1.0
35521,45,2020-01-13 17:54:20,2020-01-13 18:01:28,0001556fa4a96efdfb0e6f9c079d2fa5,17,2020-01-13,Monday,0 days 00:07:08,2020-01-13 08:51:59,188.0,-1 days +14:50:31,143.0
49077,188,2020-01-13 08:51:59,2020-01-13 08:55:04,00031615f0951f21287dc118a163dee3,8,2020-01-13,Monday,0 days 00:03:05,2020-01-13 09:00:21,189.0,0 days 00:05:17,1.0
49078,189,2020-01-13 09:00:21,2020-01-13 09:41:41,00031615f0951f21287dc118a163dee3,9,2020-01-13,Monday,0 days 00:41:20,2020-01-13 17:46:48,190.0,0 days 08:05:07,1.0
49079,190,2020-01-13 17:46:48,2020-01-13 18:25:33,00031615f0951f21287dc118a163dee3,17,2020-01-13,Monday,0 days 00:38:45,2020-01-13 18:36:26,191.0,0 days 00:10:53,1.0


In [68]:
df3 = df2[df2['seq_diff'] == 1]
df3 = df3.dropna()

In [69]:
df3.head(10)

,seq,start_timestamp,end_timestamp,vin,arrival_hour,arrival_date,arrival_day,trip_time,next_trip_start_timestamp,next_trip_seq,dwell_time,seq_diff
35516,40,2020-01-13 12:21:14,2020-01-13 12:28:51,0001556fa4a96efdfb0e6f9c079d2fa5,12,2020-01-13,Monday,0 days 00:07:37,2020-01-13 14:02:56,41.0,0 days 01:34:05,1.0
35517,41,2020-01-13 14:02:56,2020-01-13 14:10:37,0001556fa4a96efdfb0e6f9c079d2fa5,14,2020-01-13,Monday,0 days 00:07:41,2020-01-13 14:17:32,42.0,0 days 00:06:55,1.0
35518,42,2020-01-13 14:17:32,2020-01-13 14:26:23,0001556fa4a96efdfb0e6f9c079d2fa5,14,2020-01-13,Monday,0 days 00:08:51,2020-01-13 14:49:29,43.0,0 days 00:23:06,1.0
35520,44,2020-01-13 15:34:58,2020-01-13 15:38:32,0001556fa4a96efdfb0e6f9c079d2fa5,15,2020-01-13,Monday,0 days 00:03:34,2020-01-13 17:54:20,45.0,0 days 02:15:48,1.0
49077,188,2020-01-13 08:51:59,2020-01-13 08:55:04,00031615f0951f21287dc118a163dee3,8,2020-01-13,Monday,0 days 00:03:05,2020-01-13 09:00:21,189.0,0 days 00:05:17,1.0
49078,189,2020-01-13 09:00:21,2020-01-13 09:41:41,00031615f0951f21287dc118a163dee3,9,2020-01-13,Monday,0 days 00:41:20,2020-01-13 17:46:48,190.0,0 days 08:05:07,1.0
49079,190,2020-01-13 17:46:48,2020-01-13 18:25:33,00031615f0951f21287dc118a163dee3,17,2020-01-13,Monday,0 days 00:38:45,2020-01-13 18:36:26,191.0,0 days 00:10:53,1.0
52758,732,2020-01-13 10:08:07,2020-01-13 10:12:28,00089af9f7d3588a888379cf0ffc53cc,10,2020-01-13,Monday,0 days 00:04:21,2020-01-13 14:05:07,733.0,0 days 03:52:39,1.0
28932,904,2020-01-13 07:49:35,2020-01-13 08:00:47,000b56dd77e25953bc2df3b23cfb8737,7,2020-01-13,Monday,0 days 00:11:12,2020-01-13 16:05:48,905.0,0 days 08:05:01,1.0
47422,445,2020-01-13 08:38:52,2020-01-13 10:20:30,000f119dfb2d5381bc8fe3bfbfc4f4a7,8,2020-01-13,Monday,0 days 01:41:38,2020-01-13 10:24:18,446.0,0 days 00:03:48,1.0


In [92]:
dest_df = pd.read_csv(path+"viz_df2.csv")

In [93]:
dest_df = dest_df.drop(['Unnamed: 0'], axis=1).rename(columns={"Seq":'seq'})

In [94]:
dest_df.head(10)

,seq,Lat,Lon,vin
0,38,33.124920,-96.707811,0001556fa4a96efdfb0e6f9c079d2fa5
1,40,33.105293,-96.684018,0001556fa4a96efdfb0e6f9c079d2fa5
2,41,33.112107,-96.680864,0001556fa4a96efdfb0e6f9c079d2fa5
3,42,33.141374,-96.677020,0001556fa4a96efdfb0e6f9c079d2fa5
4,44,33.115048,-96.698311,0001556fa4a96efdfb0e6f9c079d2fa5
5,45,33.120333,-96.680777,0001556fa4a96efdfb0e6f9c079d2fa5
6,189,29.935532,-95.338827,00031615f0951f21287dc118a163dee3
7,190,29.975509,-95.564244,00031615f0951f21287dc118a163dee3
8,191,30.000294,-95.565431,00031615f0951f21287dc118a163dee3
9,733,41.492962,-81.995163,00089af9f7d3588a888379cf0ffc53cc


In [97]:
full_df = dest_df.merge(df3.reset_index(drop=True),on=['vin','seq'], how='left')

In [98]:
full_df.head()

,seq,Lat,Lon,vin,start_timestamp,end_timestamp,arrival_hour,arrival_date,arrival_day,trip_time,next_trip_start_timestamp,next_trip_seq,dwell_time,seq_diff
0,38,33.124920,-96.707811,0001556fa4a96efdfb0e6f9c079d2fa5,NaT,NaT,NaN,NaN,NaN,NaT,NaT,NaN,NaT,NaN
1,40,33.105293,-96.684018,0001556fa4a96efdfb0e6f9c079d2fa5,2020-01-13 12:21:14,2020-01-13 12:28:51,12.0,2020-01-13,Monday,0 days 00:07:37,2020-01-13 14:02:56,41.0,0 days 01:34:05,1.0
2,41,33.112107,-96.680864,0001556fa4a96efdfb0e6f9c079d2fa5,2020-01-13 14:02:56,2020-01-13 14:10:37,14.0,2020-01-13,Monday,0 days 00:07:41,2020-01-13 14:17:32,42.0,0 days 00:06:55,1.0
3,42,33.141374,-96.677020,0001556fa4a96efdfb0e6f9c079d2fa5,2020-01-13 14:17:32,2020-01-13 14:26:23,14.0,2020-01-13,Monday,0 days 00:08:51,2020-01-13 14:49:29,43.0,0 days 00:23:06,1.0
4,44,33.115048,-96.698311,0001556fa4a96efdfb0e6f9c079d2fa5,2020-01-13 15:34:58,2020-01-13 15:38:32,15.0,2020-01-13,Monday,0 days 00:03:34,2020-01-13 17:54:20,45.0,0 days 02:15:48,1.0


In [99]:
full_df.isna().sum()

seq                              0
Lat                              0
Lon                              0
vin                              0
start_timestamp              18432
end_timestamp                18432
arrival_hour                 18432
arrival_date                 18432
arrival_day                  18432
trip_time                    18432
next_trip_start_timestamp    18432
next_trip_seq                18432
dwell_time                   18432
seq_diff                     18432
dtype: int64

In [101]:
full_df_clean = full_df.dropna()

In [104]:
tl = [40.250580, -83.247525]
tr = [40.251652, -82.671814]
bl = [39.821981, -83.352838]
br = [39.804789, -82.633243]

In [105]:
full_df_clean = full_df_clean[(full_df_clean["Lat"] < tl[0]) & (full_df_clean["Lat"] > bl[0])]
full_df_clean = full_df_clean[(full_df_clean["Lon"] > tl[1]) & (full_df_clean["Lon"] < tr[1])]

(7121, 14)